## 导包

In [1]:
import re # 正则表达式
import numpy as np
import pandas as pd
from ToolScript.readSql import sql_tool # 数据库工具 (自行参考源代码，封装后可以针对此电脑直接调用)
from sklearn.preprocessing import OrdinalEncoder # 热度编码工具

## 数据库存写工具

In [2]:
ys = sql_tool('Yago')
read_tool = sql_tool('YagoCore')

## 需要操作的concept

In [3]:
concept_list = ['wikicat_British_economists', 'wikicat_Films_about_politicians', 'wikicat_Populated_lakeshore_places', 'wikicat_Hong_Kong_action_films']

## 读取表格

In [4]:
yagoSchema = ys.get_table('yagoSchema')
yagoDateFacts = ys.get_table('yagoDateFacts')
yagoFacts = ys.get_table('yagoFacts')
yagoLabels = ys.get_table('yagoLabels')
yagoLiteralFacts = ys.get_table('yagoLiteralFacts')
yagoTaxonomy = ys.get_table('yagoTaxonomy')
yagoTransitiveType = ys.get_table('yagoTransitiveType')

yagoSchema表读出成功!
yagoDateFacts表读出成功!
yagoFacts表读出成功!
yagoLabels表读出成功!
yagoLiteralFacts表读出成功!
yagoTaxonomy表读出成功!
yagoTransitiveType表读出成功!


In [5]:
yago_pure_tool = sql_tool('Yago_pure')
yagoCountTaxonomy = yago_pure_tool.get_table('yagoCountTaxonomy')

yagoCountTaxonomy表读出成功!


In [8]:
is_exsit_instance = yagoCountTaxonomy['concept'].values

In [6]:
subject_df = read_tool.get_table('instance_code')
predicates_df = read_tool.get_table('predicate_code')
object_3_df = read_tool.get_table('attribute_value_type_code')
object_4_df = read_tool.get_table('attribute_value_code')

instance_code表读出成功!
predicate_code表读出成功!
attribute_value_type_code表读出成功!
attribute_value_code表读出成功!


In [10]:
instance_map = subject_df.set_index(['instance'])['code'].to_dict()
predicate_map = predicates_df.set_index(['predicate'])['code'].to_dict()
object3_map = object_3_df.set_index(['attribute_value_type'])['code'].to_dict()
object4_map = object_4_df.set_index(['attribute_value'])['code'].to_dict()

**注：** 将 `DataFrame` 转换为Python内置的 `set` ，主要为了提高效率，在后面的映射编码过程中，每次查表(`DataFrame`)耗时长，具体底层不清楚，时间复杂度应该是线性 `O(n)`。而 `set` 为红黑树，时间复杂度为 `O(log n)` 。

## 定义工具类

In [24]:
class make_X_matrix:
    
    def __init__(self, concept):
        self.x = concept # 概念名称
        self.table_no = -1 # 第几个表
        self.subconcept = None
        self.subconcept_code = None
        self.instance_name = None
        self.x_index = None
        self.x_column = None
        self.instance_df = None
        self.instance_map_df = None
        self.x_matrix = None
        pass
    
    def set_table_no(self, num):
        '''设置每个概念对应的第num个矩阵，
        主要为instance_map_df中的object列不同而作区分。
        '''
        self.table_no = num
        pass
    
    def get_subconcept(self):
        '''获取concept在yagoTaxonomy表object中对应的所有行，
        逻辑为subconcept为子概念。
        '''
#         self.subconcept = yagoTaxonomy.loc[yagoTaxonomy['object'] == self.x, 'subject'].values # 此处不满足后期要求，只记录有实例的子概念
        self.subconcept = np.array([ subconcept for subconcept in yagoTaxonomy.loc[yagoTaxonomy['object'] == self.x, 'subject'].values if subconcept in is_exsit_instance])
        pass
    
    def encode_subconcept(self):
        '''对concept的子概念进行编码'''
        enc = OrdinalEncoder() # 编码工具类
        enc.fit(self.subconcept.reshape(-1, 1)) # 对子概念进行'训练'
        encode_sub = enc.transform(self.subconcept.reshape(-1, 1)) + 1 #下标从1开始
        self.subconcept_code = pd.DataFrame({'subconcept': self.subconcept, 'code': encode_sub.reshape(-1)}) # 封装成dataframe
        pass
    
    def get_instance_name(self):
        '''获取concept在yagoTransitiveType中的object中的所有行，
        即实例名。
        '''
        self.instance_name = yagoTransitiveType.loc[yagoTransitiveType['object'] == self.x, 'subject']
        pass
    
    def get_matrix_index(self):
        '''对instance_name的所有实例名按照instance_code进行映射编码。
        '''
        self.x_index = self.instance_name.apply(lambda x : subject_df.loc[subject_df['instance'] == x, 'code'].values[0]).values
        pass
    
    def filter_instance(self):
        '''获取 yagoDateFacts, yagoFacts, yagoLabels, yagoLiteralFacts 表中所有subject存在instance_name中的行。
        '''
        self.instance_df = pd.DataFrame([], columns=['subject', 'predicates', 'object'], dtype=object) # 建立表头
        tmp_list = [yagoDateFacts, yagoFacts, yagoLabels, yagoLiteralFacts] # 将3个表格封装在list中
        for tb in tmp_list: # 挨个遍历list中的table
            temp_sub = tb.loc[tb['subject'].apply(lambda x : x in self.instance_name.values), :] # 提取出在tb表中符合要求的行
            self.instance_df = pd.concat([self.instance_df, temp_sub]) # 将提取出来的行装入instance_df
        pass
    
    def map_instance_df(self):
        '''根据不同矩阵，将instance_df中的三列映射成不同的数值。
        '''
        self.instance_map_df = self.instance_df.copy()
        # 将 instance_df 拷贝到 instance_map_df，主要为了后期debug使用，同时对instance_df进行保存
        
        # 将 subject、predicates两列分别按照 instance_code、predicate_code进行映射
        self.instance_map_df.loc[:, 'subject'] = self.instance_df['subject'].apply(lambda x : instance_map[x])
        self.instance_map_df.loc[:, 'predicates'] = self.instance_df['predicates'].apply(lambda x : predicate_map[x])
        if self.table_no == 3:
            '''concept的矩阵3对于object列的映射不同于其他表，因此为特殊处理。
            此处非常耗时，可以再进行优化。主要问题是loc切片操作为线性时间复杂度
            '''
            # 前两个提取为预处理
            # 对于instance_df的object中第一个字符为'<'的object提取出来
            bracket_objects = self.instance_df.loc[
                self.instance_df['object'].apply(lambda x : x[0] == '<'), 'object'
            ].values
            
            # 将 yagoTransitiveType 的 subject在 bracket_objects 中提出出来，其实这一步应该非常耗费时间
            need_yagoTransitiveType = yagoTransitiveType.loc[
                yagoTransitiveType['subject'].apply(lambda x : x in bracket_objects), :
            ]
            
            # 下面是对instance_df中的object进行映射(根据矩阵3的具体要求)
            # 其具体要求为：如果object中的元素第一个字符为 '<'那么就去在 yagoTransitiveType 中找 subject为该object的所有行，
            # 并按照 attribute_value_type 进行映射，再将这一个或多个的映射结果按照中间加 ',' 的方式拼接起来
            obj_ = np.array([])
            # 挨个遍历 instance_df 中的每一行
            for idx, row in self.instance_df.iterrows():
                # 如果 object第一个字符为 '<'
                if row['object'][0] == '<':
                    # 就去 yagoTransitiveType 中找 subject 为 row['object'] 的所有行，
                    # 并通过 apply方法通过 attribute_value_type 映射表进行独热映射
                    tmp_ls = need_yagoTransitiveType.loc[need_yagoTransitiveType['subject'] == row['object'], 'object'].apply(
                        lambda x : object3_map[x]).values
                    # 一下三行code是对一个或多个的映射结果进行中间加 ',' 的方式拼接起来
                    _3 = '0' if len(tmp_ls) == 0 else str(int(tmp_ls[0]))
                    for i in range(1, len(tmp_ls)):
                        _3 = _3 + ',' + str(int(tmp_ls[i]))
                # 如果第一个字符不是 '<'
                else:
                    # 那么使用 object在instance表中对应的predicates找对应在 yagoSchema 中 subject 对应的object 在 attribute_value_type 对应的编码
                    _object = yagoSchema.loc[yagoSchema['subject'] == row['predicates'], 'object'].values[0]
                    _3 = str(int(object3_map[_object]))
                # 将每行的object对应的编码结果装入 obj_
                obj_ = np.append(obj_, _3)
            # 使用 obj_ 覆盖 instance_map_df 中的 object
            self.instance_map_df.loc[:, 'object'] = obj_
        else:
            # 如果不是矩阵3，
            # 那么object只需要使用attribute_value表进行映射
            self.instance_map_df.loc[:, 'object'] = self.instance_df['object'].apply(lambda x : object4_map[x])
        pass
    
    def get_matrix_columns(self):
        '''获取矩阵的列名称'''
        # 即 instance_map_df 表中的所有 predicates 的value进行去重后的结果，此处封装成np.array
        # 将去重的结果用字符进行排序(不用数值的原因在于与后面的顺序方式保持一致)，然后再转换为int类型
        self.x_column = np.sort(np.array(list(set(self.instance_map_df['predicates'].values))).astype('U25')).astype('f').astype('int')
        pass
    
    def get_matrix(self):
        '''获得最终的矩阵(除去instance、subconcept两列)
        '''
        # 首先使用 0 初始化矩阵
        self.x_matrix = pd.DataFrame(
            np.zeros((len(self.x_index), len(self.x_column)), dtype=int), 
            index=self.x_index, columns=self.x_column)
        # 利用 instance_map_df 的映射表挨个填入对应的matrix
        # instance_map_df 的subject即为矩阵的行，predicates 对应矩阵的列
        for idx, row in self.instance_map_df.iterrows():
            if self.table_no == 1:
                # 如果是矩阵1，只需要填入0、1即可
                self.x_matrix.loc[row['subject'], row['predicates']] = 1
            elif self.table_no == 2:
                # 矩阵2，如果 instance_map_df 中有一条记录，那么就需要+1
                self.x_matrix.loc[row['subject'], row['predicates']] += 1
            elif self.table_no == 3:
                # 如果是矩阵3，那么使用追加的方式，向矩阵中填入 instance_map_df 对应的 object (中间加',')
                last_val = self.x_matrix.loc[row['subject'], row['predicates']]
                # 如果 last_val 为 0 表示 是第一次往该元素格中写入元素
                if last_val == 0:
                    self.x_matrix.loc[row['subject'], row['predicates']] = row['object']
                # 在上次的基础上，中间添加','的方式追加value
                else:
                    self.x_matrix.loc[row['subject'], row['predicates']] = last_val + ',' + row['object']
            elif self.table_no == 4:
                # 如果是矩阵4，其填入的方式基本和矩阵3思路一致，仅仅是细节不同
                last_val = self.x_matrix.loc[row['subject'], row['predicates']]
                # 因为 矩阵4在 instance_map_df 映射预处理时，每个object是一个float类型，
                # 因此需要对这个value 进行 str(int(value)) 后进行拼接
                if last_val == 0:
                    self.x_matrix.loc[row['subject'], row['predicates']] = str(int(row['object']))
                else:
                    self.x_matrix.loc[row['subject'], row['predicates']] = last_val + ',' + str(int(row['object']))
        pass
    
    def append_columns(self):
        # 单独计算子概念
        # 将index添加到instance列，否则sql server中不会有该信息
        self.x_matrix['instance'] = self.x_matrix.index.values
        # 将subject列初始化为-1
        self.x_matrix['subconcept'] = int(-1)
        # 遍历x的子概念
        for sub in self.subconcept:
            # 将子概念对应在yagoTransitiveType的object的所有行的subject取出
            sub_subject_series = yagoTransitiveType.loc[yagoTransitiveType['object'] == sub, 'subject']
            
            # 遍历每一个子概念
            for son_sub in sub_subject_series:
                # 找到 subject_df 中 son_sub对应的独热编码
                idx = subject_df.loc[subject_df['instance'] == son_sub, 'code'].values[0]
                # 如果该编码在x的instance列中出现了，就需要将其子概念编码填入其中(下面的if就是在做这个事情)
                if idx in self.x_matrix.index:
                    # 编码一定唯一存在一个
                    new_value = int(self.subconcept_code.loc[self.subconcept_code['subconcept'] == sub, 'code'].values[0])
                    # 取出原来的cell当中的值
                    cell_value = self.x_matrix.loc[idx, 'subconcept']
                    # 最开始的初始化为-1，如果不等于-1说明已经填入了值，那么需要append，而不是直接覆盖
                    if cell_value != -1:
                        new_value = str(cell_value) + ',' + str(new_value)
                    
                    # 将新的val覆盖到原cell中
                    self.x_matrix.loc[idx, 'subconcept'] = new_value
        # 下面三行主要是为了调换一下x_matrix的列顺序
        new_columns = [self.x_matrix.columns.values[-2], self.x_matrix.columns.values[-1]]
        new_columns = np.append(new_columns, self.x_matrix.columns.values[:-2])
        self.x_matrix = self.x_matrix[new_columns]
        pass
    
    def set_values(self):
        '''对矩阵中的值进行去重复
        '''
        
        # 首先需要获取矩阵的位置坐标
        pos_df = self.instance_map_df[['subject', 'predicates']]
        # 所有的位置我们只需要遍历一次，所以进行一次去重
        pos_df = pos_df.loc[pos_df.duplicated() != True, :]
        
        for idx, row in pos_df.iterrows():
            str_val = str(self.x_matrix.loc[row['subject'], row['predicates']]) # 切出单个元素
            ls_val = list(set(map(int, str_val.split(',')))) # 首先利用 split进行切成单个数字，然后利用set去重
            res = str(ls_val[0])
            # 将这里list数字转换成为str
            for i in range(1, len(ls_val)):
                res = res + ',' + str(ls_val[i])
            # 填入cell
            self.x_matrix.loc[row['subject'], row['predicates']] = res
        pass
    
    def pre_work(self):
        '''对于4个矩阵来说，前4步的处理都是一样的，因此在实际处理的时候，可以只处理一次，
        即称为预处理'''
        self.get_subconcept() # 获取x的子概念
        print('完成获取子概念')
        self.encode_subconcept() # 对子概念进行编码
        print('完成子概念编码，其大小为：', self.subconcept_code.shape)
        self.get_instance_name() # 从 yagoTransitiveType 中获取所有的实例str
        print('完成获取实例名')
        self.filter_instance() # 从4个表中提取属于x实例的所有三元组
        print('完成实例筛选，其大小为：', self.instance_df.shape)
        pass
    
    def all_matrix_into_sql(self, cnt=999):
        '''将所有的矩阵存入sql server'''
        # 存入的后缀名不同
        matrix_name_list = ['', 'binary', 'number', 'type', 'value']
        self.pre_work() # 每个矩阵的相同步骤
        tool = sql_tool('YagoCore')
        table_name = 'concept_%d_subconcept_code' % cnt
        tool.to_sql(self.subconcept_code, table_name) # 将子概念的编码存入sql
        self.map_instance_df() # 映射表
        print('完成instance三元组的编码')
        self.get_matrix_index() # 获取矩阵的行名
        self.get_matrix_columns() # 获取矩阵的列名
        
        for i in [1, 2, 4, 3]: # 矩阵3的计算时间最长，放最后处理(当然可以不用这样)
            self.set_table_no(i) # 矩阵i
            if i == 3:
                # 如果是矩阵3，那么要重新计算映射表
                self.map_instance_df()
                print('最后映射完成')
            self.get_matrix()
            print('成功获取矩阵')
            self.append_columns() # 添加 instance、subconcept两列
            if i in [3, 4]:
                # 如果是矩阵3、4，那么需要对矩阵中的值进行set去重处理
                self.set_values()
            matrix_name = 'concept_' + str(cnt) + '_' + matrix_name_list[i]
            tool.to_sql(self.x_matrix, matrix_name) # 存入 sql
        
        related_table_name = 'concept_%d_related' % cnt
        tool.to_sql(self.instance_df, related_table_name)

## 执行

In [25]:
for idx in range(len(concept_list)):
    concept = '<' + concept_list[idx] + '>' # 添加尖括号，符合源数据规则
    calc_m = make_X_matrix(concept) # 创建计算矩阵的工具类
    calc_m.all_matrix_into_sql(idx + 1) # 所有矩阵放入数据库，下标从1开始

完成获取子概念
完成子概念编码，其大小为： (3, 2)


E:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:110: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_int = np.zeros((n_samples, n_features), dtype=np.int)
E:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:111: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https:

完成获取实例名
完成实例筛选，其大小为： (14098, 3)
concept_1_subconcept_code表已存入Sql Server!
完成instance三元组的编码
成功获取矩阵
concept_1_binary表已存入Sql Server!
成功获取矩阵
concept_1_number表已存入Sql Server!
成功获取矩阵
concept_1_value表已存入Sql Server!
最后映射完成
成功获取矩阵
concept_1_type表已存入Sql Server!
concept_1_related表已存入Sql Server!
完成获取子概念
完成子概念编码，其大小为： (8, 2)


E:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:110: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_int = np.zeros((n_samples, n_features), dtype=np.int)
E:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:111: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https:

完成获取实例名
完成实例筛选，其大小为： (12512, 3)
concept_2_subconcept_code表已存入Sql Server!
完成instance三元组的编码
成功获取矩阵
concept_2_binary表已存入Sql Server!
成功获取矩阵
concept_2_number表已存入Sql Server!
成功获取矩阵
concept_2_value表已存入Sql Server!
最后映射完成
成功获取矩阵
concept_2_type表已存入Sql Server!
concept_2_related表已存入Sql Server!
完成获取子概念
完成子概念编码，其大小为： (5, 2)


E:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:110: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_int = np.zeros((n_samples, n_features), dtype=np.int)
E:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:111: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https:

完成获取实例名
完成实例筛选，其大小为： (32804, 3)
concept_3_subconcept_code表已存入Sql Server!
完成instance三元组的编码
成功获取矩阵
concept_3_binary表已存入Sql Server!
成功获取矩阵
concept_3_number表已存入Sql Server!
成功获取矩阵
concept_3_value表已存入Sql Server!
最后映射完成
成功获取矩阵
concept_3_type表已存入Sql Server!
concept_3_related表已存入Sql Server!
完成获取子概念
完成子概念编码，其大小为： (4, 2)


E:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:110: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_int = np.zeros((n_samples, n_features), dtype=np.int)
E:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:111: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https:

完成获取实例名
完成实例筛选，其大小为： (8844, 3)
concept_4_subconcept_code表已存入Sql Server!
完成instance三元组的编码
成功获取矩阵
concept_4_binary表已存入Sql Server!
成功获取矩阵
concept_4_number表已存入Sql Server!
成功获取矩阵
concept_4_value表已存入Sql Server!
最后映射完成
成功获取矩阵
concept_4_type表已存入Sql Server!
concept_4_related表已存入Sql Server!


## 完成